# Simple Risk Quantification

Please see the Riskquant blog if you want detailed info on the simulation code:  
https://medium.com/@NetflixTechBlog/open-sourcing-riskquant-a-library-for-quantifying-risk-6720cc1e4968


# Step 1:

## Update the Loss Fields

These are sample values only. Update these fields with values from your internal loss tables.

In [1]:
# Dollar value of one hour of one employee
EMPLOYEE_HOURLY_COST = 50

# Lowest cost to respond to an incident
LOW_RESPONSE_COST = 100
# Highest cost to respond to an incident
HIGH_RESPONSE_COST = 10000

# Lowest cost of PII data loss
LOW_PII_COST = 10000
# Highest cost of PII data loss
HIGH_PII_COST = 1000000

# Lowest cost of PCI data loss
LOW_PCI_COST = 100000
# Highest cost of PII data loss
HIGH_PCI_COST = 10000000


# Step 2

## Run the notebook

 1. Click Cell > Run All
 2. Scroll down to the bottom of this notebook
 3. Under **Risk Quantification** make selections based on your scenario, and click Simulate Risk

In [2]:
# If you have not yet installed riskquant, install it
# %pip install --upgrade riskquant

In [3]:
# You can change the code here but you shouldn't need to... 
# Cnce you've clicked Cell > Run all, scroll to the bottom of this cell for the interactive section
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from matplotlib import pyplot as plt
from matplotlib import ticker as mtick
import numpy as np
from riskquant import pertloss

def loss_exceedance_curve(simulate,
                          title="Aggregated Loss Exceedance",
                          xlim=[100000, 1000000000]):
    losses = np.array([np.percentile(simulate, x) for x in range(1, 100, 1)])
    percentiles = np.array([float(100 - x) / 100.0 for x in range(1, 100, 1)])
    _ = plt.figure()
    ax = plt.gca()
    ax.plot(losses, percentiles)
    plt.title(title)
    ax.set_xscale("log")
    ax.set_ylim(0.0, percentiles[np.argmax(losses > 0.0)] + 0.05)
    ax.set_xlim(xlim[0], xlim[1])
    xtick = mtick.StrMethodFormatter('${x:,.0f}')
    ax.xaxis.set_major_formatter(xtick)
    ytick = mtick.StrMethodFormatter('{x:.000%}')
    ax.yaxis.set_major_formatter(ytick)
    plt.grid(which='both')
    plt.show()

# Frequency widgets
frequency_menu = widgets.Dropdown(
       options=['Once per Day',
                'Once per Week',
                'Once per Month',
                'Once per Three Months',
                'Once per Year',
                'Once per Two Years',
                'Once per Ten Years',
                'Once per One Hundred Years'],
       style={'description_width': '300px'},
       layout={'width': '500px'},
       description='Loss Event Frequency:')

# Magnitude widgets
emp = widgets.Text(value='0',description='Number of employees resolving scenario', 
                   style={'description_width': '300px'},
                   layout={'width': '400px'} )
emp_hours = widgets.Text(value='0',description='Number of hours each employee will spend',
                         style={'description_width': '300px'},
                         layout={'width': '400px'}  )
pci_checkbox = widgets.Checkbox(description='Scenario includes loss of PCI data',
                                layout={'width': '400px'} )
pii_checkbox = widgets.Checkbox(description='Scenario includes loss of PII data',
                                layout={'width': '400px'})

risk_button = widgets.Button(description='Simulate Risk')

out = widgets.Output()

def on_button_clicked(b):
    with out:
        clear_output() 
        freq_input = frequency_menu.value

        if freq_input == 'Once per Day':
            minimum_frequency = 310
            maximum_frequency = 420
            most_likely_frequency = 365
        elif freq_input == 'Once per Week':
            minimum_frequency = 44
            maximum_frequency = 60
            most_likely_frequency = 52   
        elif freq_input == 'Once per Month':
            minimum_frequency = 8
            maximum_frequency = 16
            most_likely_frequency = 12 
        elif freq_input == 'Once per Three Months':
            minimum_frequency = 2
            maximum_frequency = 6
            most_likely_frequency = 4 
        elif freq_input == 'Once per Year':
            minimum_frequency = 0.5
            maximum_frequency = 2
            most_likely_frequency = 1 
        elif freq_input == 'Once per Two Years':
            minimum_frequency = 0.3
            maximum_frequency = 1
            most_likely_frequency = 0.5 
        elif freq_input == 'Once per Ten Years':
            minimum_frequency = 0.05
            maximum_frequency = 0.25
            most_likely_frequency = 0.1 
        elif freq_input == 'Once per One Hundred Years':
            minimum_frequency = 0.005
            maximum_frequency = 0.015
            most_likely_frequency = 0.01 
        else:
            minimum_frequency = 0.05
            maximum_frequency = 0.25
            most_likely_frequency = 0.1 

        if pci_checkbox.value and pii_checkbox.value:
            low_loss = LOW_PCI_COST + LOW_PII_COST  
            high_loss = HIGH_PCI_COST + HIGH_PII_COST
        elif pci_checkbox.value and pii_checkbox.value == False:
            low_loss = LOW_PCI_COST
            high_loss = HIGH_PCI_COST    
        elif pci_checkbox.value == False and pii_checkbox.value:
            low_loss = LOW_PII_COST
            high_loss = HIGH_PII_COST  
        else:
            low_loss = LOW_RESPONSE_COST
            high_loss = HIGH_RESPONSE_COST      


        try:
            emp_int = int(emp.value)
        except:
            print('Please enter an integer for number of employees')

        try:
            emp_hours_int = int(emp_hours.value)
        except:
            print('Please enter an integer for number of employees')

        emp_hour_cost =  emp_hours_int * EMPLOYEE_HOURLY_COST
        emp_cost = emp_hour_cost * emp_int

        high_loss += emp_cost

        p = pertloss.PERTLoss(low_loss=low_loss, 
                              high_loss=high_loss, 
                              min_freq=minimum_frequency, 
                              max_freq=maximum_frequency, 
                              most_likely_freq=most_likely_frequency)
        simulate = p.simulate_years(50000)
        loss_summary = p.summarize_loss(simulate)
        annualized_loss_exposure = loss_summary['ninetieth_percentile'] * most_likely_frequency
        print('Annualized Loss Exposure (worst case):\n')
        ale = round((most_likely_frequency * loss_summary['ninetieth_percentile']), -3)
        print('$', ale )
        print('\nLoss Event Frequency:\n')
        print(freq_input)
        print('\nLoss summary:')
        print(loss_summary)
        loss_exceedance_curve(simulate)


risk_button.on_click(on_button_clicked)
info = Markdown("""# Risk Quantification

**This is a starting point.  Please work with your Risk team for a detailed Risk Quantification.**

**This is only a simulation based on the loss information entered.**

**Instructions:**
- Select a Loss Event Frequency: The frequency within the next year that loss will materialize from a threat agent’s action.
- Enter the number of employees involved in responding to the scenario.
- Enter the number of hours each employee will spend responding to the scenario.
- Select if the scenario includes PCI and/or PII data.
- Click **Simulate Risk**""")

grapher = widgets.VBox([frequency_menu, emp, emp_hours, pci_checkbox, 
                        pii_checkbox, risk_button, out])
display(info, grapher)

# Risk Quantification

**This is a starting point.  Please work with your Risk team for a detailed Risk Quantification.**

**This is only a simulation based on the loss information entered.**

**Instructions:**
- Select a Loss Event Frequency: The frequency within the next year that loss will materialize from a threat agent’s action.
- Enter the number of employees involved in responding to the scenario.
- Enter the number of hours each employee will spend responding to the scenario.
- Select if the scenario includes PCI and/or PII data.
- Click **Simulate Risk**